# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

Description of project: Cournot duopoly

Equations:
Aggregate demand
Individual profit functions (used to find aggregate supply)

Analytical solution:
Find individual response functions

Grid search:
Try different outputs and check that supply equates demand.

Extensions:
Add more periods -> Easier to collude
Add more firms -> Harder to collude
Vary delta -> see how collusion is harder to sustain
Vary homogeniety of products
Vary costs - draw from distribution
Entry/exit
Simulate model over time - plot quantity/price for different shocks to parameters


Set up as class?

In [82]:
import numpy as np
import sympy as sm

from scipy import optimize
from numpy import array
import matplotlib.pyplot as plt

%matplotlib inline
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Demand and cost

In [83]:
# Defining symbolic variables for the optimization problem
x = sm.symbols('x')  # Symbolic variable representing x
x_rest = sm.symbols('x_rest')  # Symbolic variable representing x_rest

# Defining the objective function
objective = (1 - x - x_rest - c) * x

# Taking the first derivative of the objective function w.r.t x
obj_dif = sm.diff(objective, x)

# Converting the symbolic expression for the derivative into a callable function
best = sm.lambdify(args=(x, x_rest), expr=obj_dif)

# Taking the second derivative of the objective function w.r.t x
best_dif = sm.diff(obj_dif, x)

# Converting the symbolic expression for the second derivative into a callable function
jac_x = sm.lambdify(args=(x, x_rest), expr=best_dif)

# Taking the second derivative of the objective function w.r.t x_rest
best_dif = sm.diff(obj_dif, x_rest)

# Converting the symbolic expression for the second derivative into a callable function
jac_x_rest = sm.lambdify(args=(x, x_rest), expr=best_dif)

In [85]:
# Setting up the parameters for the optimization problem
c = 0  # Constant value for the objective function
N = 100  # Number of variables

# Setting up the initial values for x_rest
x_rest = np.zeros(N)

# Defining the function to be optimized
def h(x):
    y = np.zeros(N)
    for i in range(N):
        # Evaluating the first derivative of the objective function at x[i]
        y[i] = best(x[i], sum(x) - x[i])
    return y

# Defining the Jacobian of the function to be optimized
def hp(x):
    y = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if j == i:
                # Evaluating the second derivative of the objective function w.r.t x[i]
                y[i,j] = jac_x(x[i], sum(x) - x[i])
            else:
                # Evaluating the second derivative of the objective function w.r.t x_rest[i]
                y[i,j] = jac_x_rest(x[i], sum(x) - x[i])
    return y

# Setting up the initial values for x
x0 = np.array(x_rest)

# Solving the optimization problem using scipy.optimize.root() function
result = optimize.root(h, x0, jac=hp)

# Printing the results
print(result)
print('\nx =', result.x, ', h(x) =', h(result.x))

 message: The solution converged.
 success: True
  status: 1
     fun: [ 1.110e-16  3.331e-16 ...  1.110e-16  1.110e-16]
       x: [ 9.901e-03  9.901e-03 ...  9.901e-03  9.901e-03]
    nfev: 3
    njev: 1
    fjac: [[-1.971e-01 -9.853e-02 ... -9.853e-02 -9.853e-02]
           [ 6.951e-01 -7.157e-01 ... -6.882e-03 -6.882e-03]
           ...
           [ 1.005e-02  1.005e-02 ... -9.950e-01 -9.952e-05]
           [ 9.951e-03  9.951e-03 ...  9.951e-03 -9.951e-01]]
       r: [ 1.015e+01  1.005e+01 ...  1.015e-02  1.005e+00]
     qtf: [ 9.714e-15 -6.594e-16 ... -7.142e-17 -7.071e-17]

x = [0.00990099 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099
 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099
 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099
 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099
 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099
 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099 0.00990099
 0.00990

In [6]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 12})
import ipywidgets as widgets

from scipy import linalg
from scipy import optimize

#Identical good
c=1

def inv_demand(x,x_rest):
    return 1-x-x_rest

#Identical production functions
def cost(x,c=0.1):
    if x == 0:
        cost_value = 0
    else:
        cost_value = c*x
    return cost_value

def revenue(x,x_rest):
    return inv_demand(x,x_rest)*x

def profit(x,x_rest,c):
    return revenue(x,x_rest)-cost(x,c)

def solve(x_rest,c):
    obj= lambda x: -profit(x,x_rest,c)
    solution = optimize.minimize_scalar(obj, method='bounded', bounds=(0,1))
    return solution

solve(0.1,0.5)

 message: Solution found.
 success: True
  status: 0
     fun: -0.03999999999999998
       x: 0.20000000000000012
     nit: 6
    nfev: 6

# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 